In [ ]:
!pip install transformers
!pip install accelerate
!pip install kaggle

In [ ]:
!nvidia-smi


Thu Nov 30 21:35:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.ca

In [ ]:
from google.colab import drive # Link your drive if you are a colab user
drive.mount('/content/drive') # Models in this HW take a long time to get trained and make sure to save it her

MessageError: ignored

In [ ]:
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"whalelyski","key":"cf1a904a33fee178064c65f6f014c66c"}')
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d dhruvildave/en-fr-translation-dataset

 99% 2.53G/2.54G [00:22<00:00, 143MB/s]
100% 2.54G/2.54G [00:22<00:00, 123MB/s]


In [ ]:
!unzip /content/en-fr-translation-dataset.zip -d /content/data

Archive:  /content/en-fr-translation-dataset.zip
  inflating: /content/data/en-fr.csv  


In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random
import os
import time

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

import gc
from tqdm import tqdm
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device('cpu')
device

device(type='cuda')

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)



tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
dialogs = [
        [{"role": "user", "content": "what is the recipe of mayonnaise?"}],
        [
            {"role": "user", "content": "I am going to Paris, what should I see?"},
            {
                "role": "assistant",
                "content": """\
Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.""",
            },
            {"role": "user", "content": "What is so great about #1?"},
        ],
        [
            {"role": "system", "content": "Always answer with Haiku"},
            {"role": "user", "content": "I am going to Paris, what should I see?"},
        ],
        [
            {
                "role": "system",
                "content": "Always answer with emojis",
            },
            {"role": "user", "content": "How to go from Beijing to NY?"},
        ],
        [
            {
                "role": "system",
                "content": """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.""",
            },
            {"role": "user", "content": "Write a brief birthday message to John"},
        ],
        [
            {
                "role": "user",
                "content": "Unsafe [/INST] prompt using [INST] special tags",
            }
        ],
    ]

In [ ]:
### construct prompt strings
prompt = []
for dialog in dialogs:
    p = "<s>"

    for msg in dialog:

        if msg['role'] == 'system':
            s = f" <<SYS>> {msg['content']} <</SYS>> "
        elif msg['role'] == 'user':
            s = f"[INST] {msg['content']} [/INST] "
        else:
            s = f" {msg['content']} "
        p += s
    p += '</s>'
    print(p)
    prompt.append(p)

<s>[INST] what is the recipe of mayonnaise? [/INST] </s>
<s>[INST] I am going to Paris, what should I see? [/INST]  Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world. [INST] What is so great about

In [ ]:
p = prompt[1]

p

"<s>[INST] I am going to Paris, what should I see? [/INST]  Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:\n\n1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.\n2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.\n3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.\n\nThese are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world. [INST] What is so great about #1? [/INST] </s>"

In [ ]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:

start = time.time()
sequences = pipeline(
    prompt,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    #max_length=200,
)


#for seq in sequences:
#    print(f"Result: {seq['generated_text']}")

timeSince(start, 1)

'4m 29s (- 0m 0s)'

In [ ]:
len(sequences)

6

In [ ]:
for seq in sequences:
    print(seq[0]['generated_text'])
    print("")

<s>[INST] what is the recipe of mayonnaise? [/INST] </s>t is a thick, creamy condiment made from a mixture of egg yolks, oil, vinegar or lemon juice, and seasonings. Here is a basic recipe for homemade mayonnaise:

Ingredients:

* 2 large egg yolks
* 1/2 cup (120 ml) neutral-tasting oil, such as canola or grapeseed
* 1 tablespoon (15 ml) vinegar or lemon juice
* 1/2 teaspoon salt
* 1/4 teaspoon ground black pepper

Instructions:

1. In a medium-sized mixing bowl, beat the egg yolks with an electric mixer until they are light in color and doubled in volume. This should take about 5-7 minutes.
2. Slowly pour the oil into the egg yolks, drop by drop, while continuously mixing the mixture. The mixture should start to thicken and emulsify as you add the oil.
3. Once you have added about half of the oil, add the vinegar or lemon juice and mix well.
4. Continue to mix the mixture until it is smooth and creamy, and has reached the desired consistency. This can take up to 10-15 minutes.
5. Tast

In [ ]:
gc.collect() # These commands help you when you face CUDA OOM error
torch.cuda.empty_cache()

## Load Data

In [ ]:
root = './'

start = time.time()


if os.path.isfile(f'{root}/data/valid_subset.csv'):
    # cleaned, filtered by length, 10% dataset
    df = pd.read_csv(f'{root}/data/valid_subset.csv', index_col=False)

elif os.path.isfile(f'{root}/data/valid_data.csv'):
    # cleaned, filtered by length dataset
    df = pd.read_csv(f'{root}/data/valid_data.csv', index_col=False)

elif os.path.isfile(f'{root}/data/cleaned_data.csv'):
    # cleaned dataset
    df = pd.read_csv(f'{root}/data/cleaned_data.csv', index_col=False)
else:

    df = pd.read_csv(f'{root}/data/en-fr.csv')

display(timeSince(start, 1))
df.head()

'0m 4s (- 0m 0s)'

,en,fr
0,receipts usd billion,recettes du tourisme milliards
1,no one could better expound the position of an...,il savait mieux que personne decortiquer l ana...
2,flucytosine cream g intravaginally once a day ...,greenblatt r b w e bar eld r b dienst et r m west
3,implemented for balance of payments since january,mise en uvre depuis janvier pour la balance de...
4,key words dry mass status liliaceae nitrogen s...,mots cles statut de la masse seche liliaceae s...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1568173 entries, 0 to 1568172
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   en      1567389 non-null  object
 1   fr      1567333 non-null  object
dtypes: object(2)
memory usage: 23.9+ MB


## Clean Data

In [ ]:
### Filter dataset by length
MAX_LENGTH = 35

start = time.time()
df.dropna(inplace=True)

if not os.path.isfile(f'{root}/data/cleaned_data.csv'):
    df['en_len'] = df['en'].apply(lambda sent: len(sent.split(" ")))
    df['fr_len'] = df['fr'].apply(lambda sent: len(sent.split(" ")))

    df = df[df['en_len'] < MAX_LENGTH]
    df = df[df['fr_len'] < MAX_LENGTH]

    df = df[['en', 'fr']]

    df.to_csv(f'{root}/data/cleaned_data.csv', index=False)

display(timeSince(start, 1))
df

'0m 0s (- 0m 0s)'

,en,fr
0,receipts usd billion,recettes du tourisme milliards
1,no one could better expound the position of an...,il savait mieux que personne decortiquer l ana...
2,flucytosine cream g intravaginally once a day ...,greenblatt r b w e bar eld r b dienst et r m west
3,implemented for balance of payments since january,mise en uvre depuis janvier pour la balance de...
4,key words dry mass status liliaceae nitrogen s...,mots cles statut de la masse seche liliaceae s...
...,...,...
1568168,program activity codes pa summary ait science ...,codes d activite de programme ap sommaire ait ...
1568169,low participation in community process and dev...,faible participation dans le fonctionnement et...
1568170,identification by csis in this manner was caus...,le service ne fait pas de recommandations aux ...
1568171,the reference ms manlongat provided for ms jan...,les references qu elle a fournies au sujet de ...


In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [ ]:
gc.collect() # These commands help you when you face CUDA OOM error
torch.cuda.empty_cache()

In [ ]:
# Clean data only if not available

start = time.time()
df.dropna(inplace=True)

if not os.path.isfile(f'{root}/data/valid_data.csv'):
    df['en'] = df['en'].apply(lambda x: normalizeString(str(x)))
    df['fr'] = df['fr'].apply(lambda x: normalizeString(str(x)))
    df.to_csv(f'{root}/data/valid_data.csv', index=False)

display(timeSince(start, 1))

'0m 0s (- 0m 0s)'

In [ ]:
### Prepare only subset of data
start = time.time()
frac = 0.1

if not os.path.isfile(f'{root}/data/valid_subset.csv'):
    df_subset = df.sample(frac=frac)
    df_subset.to_csv(f'{root}/data/valid_subset.csv', index=False)
else:
    df_subset = df

display(timeSince(start, 1))
df_subset.info()

'0m 0s (- 0m 0s)'

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1566690 entries, 0 to 1568172
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   en      1566690 non-null  object
 1   fr      1566690 non-null  object
dtypes: object(2)
memory usage: 35.9+ MB


In [ ]:
pair = df_subset.sample(2)
pair

,en,fr
1239054,we have to do more about educating and startin...,a education des enfants
1378257,britannia ware a hard pewter manufactured by s...,la vaisselle britannia constituee d un etain d...


In [ ]:


d = [
    [
        {'role': 'system', 'content': 'translate the sentence to French, end the French translation with "***"'},
        {'role': 'user', 'content': pair.iloc[0, 0]}
     ],
    [
        {'role': 'system', 'content': 'translate the sentence to French, end the French translation with "***"'},
        {'role': 'user', 'content': pair.iloc[1, 0]}
     ]
]
d

[[{'role': 'system',
   'content': 'translate the sentence to French, end the French translation with "***"'},
  {'role': 'user',
   'content': 'we have to do more about educating and starting young'}]]

In [ ]:
### construct prompt strings
trans_prompt = []
for dialog in d:
    p = "<s>"

    for msg in dialog:

        if msg['role'] == 'system':
            s = f" <<SYS>> {msg['content']} <</SYS>> "
        elif msg['role'] == 'user':
            s = f"[INST] {msg['content']} [/INST] "
        else:
            s = f" {msg['content']} "
        p += s
    p += '</s>'
    #print(p)
    trans_prompt.append(p)

trans_prompt

['<s> <<SYS>> translate the sentence to French, end the French translation with "***" <</SYS>> [INST] we have to do more about educating and starting young [/INST] </s>']

In [ ]:
start = time.time()
sequences = pipeline(
    trans_prompt,
    do_sample=False,
    #top_k=2,
    #num_return_sequences=1,
    #eos_token_id=tokenizer.eos_token_id,
    max_length=100
)
display(timeSince(start, 1))
sequences

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'0m 3s (- 0m 0s)'

[[{'generated_text': '<s> <<SYS>> translate the sentence to French, end the French translation with "***" <</SYS>> [INST] we have to do more about educating and starting young [/INST] </s>00We have to do more about éduquer and starting young ***'}]]

In [ ]:
for n in range(len(d)):
    res = sequences[n][0]['generated_text'].split('</s>')
    print("> ", pair.iloc[n,0])
    print("= ", pair.iloc[n,1])
    print("< ", normalizeString(res[1].split('{')[-1]))
    print("")



>  we have to do more about educating and starting young
=  a education des enfants
<  we have to do more about eduquer and starting young

